# Script to pull income statement data from HTML tags on Yahoo! Finance and store it for use in an Excel model.


### Set up imports and request headers

In [7]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

# set up request headers to simulate browser request

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'close',
    'DNT': '1', # Do Not Track Request Header 
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

### Set up target company and generate financial statement + cash flow URLs

In [8]:
# select target company to evaluate and get its statement URLs

symbol =  'CRSP'
print(symbol)#DEBUG

fin_url = f'https://finance.yahoo.com/quote/{symbol}/financials?p={symbol}'
cf_url = f'https://finance.yahoo.com/quote/{symbol}/cash-flow?p={symbol}'
print(fin_url)#DEBUG
print (cf_url)#DEBUG

CRSP
https://finance.yahoo.com/quote/CRSP/financials?p=CRSP
https://finance.yahoo.com/quote/CRSP/cash-flow?p=CRSP


### Fetch each page using requests, and parse them using lxml

In [10]:
### find target company income statements

# fetch the page
fin_page = requests.get(fin_url, headers=headers)
print(fin_page)#DEBUG

# parse the page with lxml
fin_tree = html.fromstring(fin_page.content)
print(fin_tree)#DEBUG

fin_tree.xpath("//h1/text()")#DEBUG

### find target company cash flows

# fetch the page
cf_page = requests.get(cf_url, headers=headers)
print(cf_page)#DEBUG

# parse the page with lxml
cf_tree = html.fromstring(cf_page.content)
print(cf_tree)#DEBUG

cf_tree.xpath("//h1/text()")#DEBUG

<Response [200]>
<Element html at 0x196206a5c70>
<Response [200]>
<Element html at 0x1961d83d360>


['CRISPR Therapeutics AG (CRSP)']

## Create table for financial statement data

In [16]:
# make rows variable assigned to html tags with class D(tbr) -- tablerow
fin_table_rows = fin_tree.xpath("//div[contains(@class, 'D(tbr)')]")    

# make sure more than 0 rows
assert len(fin_table_rows) > 0

# table variable
parsed_rows = []

# for each row in our parsed HTML data
for table_row in fin_table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    # for each element in the row
    for rs in el:
        # append the value to our table
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        # or append an N/A value
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    # if the whole row is empty, add to table
    if (none_count < 4):
        parsed_rows.append(parsed_row)

# Make a DataFrame with our parsed data
fin_df = pd.DataFrame(parsed_rows)
fin_df

,0,1,2,3,4,5
0,Breakdown,ttm,12/31/2022,12/31/2021,12/31/2020,12/31/2019
1,Total Revenue,"170,100",436,"913,081",543,"289,590"
2,Cost of Revenue,"132,510","110,250",NaN,NaN,NaN
3,Gross Profit,"37,590","-109,814",NaN,NaN,NaN
4,Operating Expense,"511,229","563,347","539,553","354,978","242,850"
5,Operating Income,"-473,639","-673,161","373,528","-354,435","46,740"
6,Other Income Expense,"49,902","22,661","6,003","6,379","20,566"
7,Pretax Income,"-423,737","-650,500","379,531","-348,056","67,306"
8,Tax Provision,"-7,808",-325,"1,870",809,448
9,Net Income Common Stockholders,"-415,929","-650,175","377,661","-348,865","66,858"


## Create table for cash flow data

In [13]:
# make rows variable assigned to html tags with class D(tbr) -- tablerow
cf_table_rows = cf_tree.xpath("//div[contains(@class, 'D(tbr)')]")

# make sure more than 0 rows
assert len(cf_table_rows) > 0

# table variable
parsed_rows = []

# for each row in our parsed HTML data
for table_row in cf_table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    # for each element in the row
    for rs in el:
        # append the value to our table
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        # or append an N/A value
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    # if the whole row is empty, add to table
    if (none_count < 4):
        parsed_rows.append(parsed_row)

# Make a DataFrame with our parsed data
cf_df = pd.DataFrame(parsed_rows)
cf_df

,0,1,2,3,4,5
0,Breakdown,ttm,12/31/2022,12/31/2021,12/31/2020,12/31/2019
1,Operating Cash Flow,"-342,992","-495,741","538,972","-238,366","56,677"
2,Investing Cash Flow,"246,684","-258,655","-1,035,430","-541,170","1,325"
3,Financing Cash Flow,"39,759","38,592","250,945","1,016,152","430,983"
4,End Cash Position,"456,520","224,060","939,944","1,185,468","948,812"
5,Capital Expenditure,"-18,822","-37,188","-81,705","-18,358","-6,684"
6,Issuance of Capital Stock,NaN,970,"213,267","982,289","415,019"
7,Repurchase of Capital Stock,NaN,NaN,NaN,0,0
8,Free Cash Flow,"-361,814","-532,929","457,267","-256,724","49,993"
